## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件
对于提供的数据集，不要改存储地方，也不要修改文件名和内容
不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块
写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！
这次作业很重要，一定要完成！ 相信会有很多的收获！

# 导入相关的包 


In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics
# from sklearn.externals import joblib
from bayes_opt import BayesianOptimization
from gensim import models
import joblib
import re
import jieba
# from scipy.special import softmax
from sklearn.grid_search import RandomizedSearchCV

In [2]:
max_length = 500  # 表示样本表示最大的长度,表示降维之后的维度
sentence_max_length = 1500  # 表示句子/样本在降维之前的维度
Train_features3, Test_features3, Train_label3, Test_label3 = [], [], [], []

# 读取训练好的词嵌入向量和文本文件

## KeyedVectors.load_word2vec_format(file path, binary)  
file path: 词向量模型路径  
binary: text format or binary format

It turns out that load_word2vec_format is used when we're trying to load word vectors that are trained using the original implementation of word2vec (in C). Since these pre-trained word vectors are trained using Python (gensim), we can use load instead.

In [3]:
# ToDo
# 通过models.KeyedVectors加载预训练好的embedding
# fast_embedding = models.KeyedVectors.load_word2vec_format("fast.model", binary=False, unicode_errors='ignore')
# w2v_embedding = models.KeyedVectors.load_word2vec_format("w2v.model", binary=False, unicode_errors='ignore')
print('load w2v_embedding model')
w2v_embedding =  models.Word2Vec.load('w2v.model')  # Todo
print('load fast_embedding model')
fast_embedding = models.FastText.load('fast.model')  # Todo
print("fast_embedding输出词表的个数{},w2v_embedding输出词表的个数{}".format(
    len(fast_embedding.wv.vocab.keys()), len(w2v_embedding.wv.vocab.keys())))

print("取词向量成功")

load w2v_embedding model
load fast_embedding model
fast_embedding输出词表的个数116571,w2v_embedding输出词表的个数19592
取词向量成功


In [4]:
# ToDo
# 读取train_clean.tsv ，test_clean.tsv训练集和测试集文件
# hint: 通过pandas中的read_csv读取数据集
train = pd.read_csv('train_clean.csv', sep='\t')
test = pd.read_csv('test_clean.csv', sep='\t')


# train = train[0:1000]
# test = test[0:1000]

print("读取数据完成")

读取数据完成


In [5]:
train.label.unique()

array(['文化', '文学', '管理', '社会科学', '小说', '大中专教材教辅', '艺术', '外语学习', '建筑',
       '旅游/地图', '励志与成功', '童书', '医学', '科学与自然', '工业技术', '中小学教辅', '历史', '考试',
       '青春文学', '哲学/宗教', '法律', '计算机与互联网', '育儿/家教', '金融与投资', '政治/军事',
       '健身与保健', '孕产/胎教', '农业/林业', '国学/古籍', '动漫', '烹饪/美食', '科普读物', '婚恋与两性'],
      dtype=object)

In [6]:
label_list = []
for l in train.label.unique():
    l_ = list(jieba.cut(l, cut_all=False))
    label_list.append(l_)

label_list

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.664 seconds.
Prefix dict has been built successfully.


[['文化'],
 ['文学'],
 ['管理'],
 ['社会科学'],
 ['小说'],
 ['大中专', '教材', '教辅'],
 ['艺术'],
 ['外语', '学习'],
 ['建筑'],
 ['旅游', '/', '地图'],
 ['励志', '与', '成功'],
 ['童书'],
 ['医学'],
 ['科学', '与', '自然'],
 ['工业', '技术'],
 ['中小学', '教辅'],
 ['历史'],
 ['考试'],
 ['青春文学'],
 ['哲学', '/', '宗教'],
 ['法律'],
 ['计算机', '与', '互联网'],
 ['育儿', '/', '家教'],
 ['金融', '与', '投资'],
 ['政治', '/', '军事'],
 ['健身', '与', '保健'],
 ['孕产', '/', '胎教'],
 ['农业', '/', '林业'],
 ['国学', '/', '古籍'],
 ['动漫'],
 ['烹饪', '/', '美食'],
 ['科普读物'],
 ['婚恋', '与', '两性']]

In [7]:
useless_words = ['/', '与']
for l in label_list:
    for w in useless_words:
        if w in l:
            l.remove(w)
len(label_list)

33

In [8]:
label_list

[['文化'],
 ['文学'],
 ['管理'],
 ['社会科学'],
 ['小说'],
 ['大中专', '教材', '教辅'],
 ['艺术'],
 ['外语', '学习'],
 ['建筑'],
 ['旅游', '地图'],
 ['励志', '成功'],
 ['童书'],
 ['医学'],
 ['科学', '自然'],
 ['工业', '技术'],
 ['中小学', '教辅'],
 ['历史'],
 ['考试'],
 ['青春文学'],
 ['哲学', '宗教'],
 ['法律'],
 ['计算机', '互联网'],
 ['育儿', '家教'],
 ['金融', '投资'],
 ['政治', '军事'],
 ['健身', '保健'],
 ['孕产', '胎教'],
 ['农业', '林业'],
 ['国学', '古籍'],
 ['动漫'],
 ['烹饪', '美食'],
 ['科普读物'],
 ['婚恋', '两性']]

In [9]:
label_embedding = []

for l in label_list:
    label_vector = []
    for w in l:
        label_vector.append(fast_embedding[w])
    label_embedding.append(list(np.mean(label_vector, axis=0)))

label_embedding = np.asarray(label_embedding)
label_embedding.shape

/users/home/jwu51/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


(33, 300)

将df中的label映射为数字标签并保存到labelIndex列中

In [10]:
labelName = train.label.unique()  # 全部label列表
labelIndex = list(range(len(labelName)))  # 全部label标签
labelNameToIndex = dict(zip(labelName, labelIndex))  # label的名字对应标签的字典
labelIndexToName = dict(zip(labelIndex, labelName))  # label的标签对应名字的字典
train["labelIndex"] = train.label.map(labelNameToIndex)
test["labelIndex"] = test.label.map(labelNameToIndex)

In [11]:
def query_cut(query):
    '''
    函数说明：该函数用于对输入的语句（query）按照空格进行切分
    '''
    # ToDo
    # 第一步：定义一个query_cut函数 将query按空格划分并返回，
    # hint: 通过sqlit对query进行划分
    query_list = query.split(" ")
    return query_list

# 第二步：然后train和test中的每一个样本都调用该函数，
#      并将划分好的样本分别存储到train["queryCut"]和test["queryCut"]中

train_list = []
test_list = []

for i in range(len(list(train.text))):
    train_list.append(query_cut(train.text[i]))
    
for i in range(len(test)):
    test_list.append(query_cut(test.text[i]))    
    
train["queryCut"] = train_list
test["queryCut"] = test_list
print("切分数据完成")

切分数据完成


In [12]:
with open('stopwords.txt', "r", encoding="utf8") as f:
    #ToDo
    # 第一步：按行读取停用词文件
    stopWords = [line.rstrip() for line in f]


In [13]:
def rm_stop_word(wordList):
    '''
    函数说明：该函数用于去除输入样本中的存在的停用词
    Return: 返回去除停用词之后的样本
    '''
    # ToDo
    # 第二步：去除每个样本中的停用词并返回新的样本
    return [w for w in wordList if w not in stopWords and not re.match('^[a-z|A-Z|0-9|.]*$',w)]
    
train["queryCutRMStopWord"] = train["queryCut"].apply(rm_stop_word)
# dev["queryCutRMStopWord"] = dev["text"].apply(rm_stop_word)
test["queryCutRMStopWord"] = test["queryCut"].apply(rm_stop_word)
print("去除停用词")


去除停用词


In [14]:
train["queryCutRMStopWord"]

0         [杏林, 芳菲, 广东, 中医药, 曹磊, 编著, 杏林, 芳菲, 广东, 中医药, 注重,...
1         [额吉, 白云, 优秀, 蒙古文, 文学作品, 翻译, 出版, 工程, 第五辑, 散文, 卷...
2         [叶赛宁, 抒情诗, 选, 本书, 收入, 叶赛宁, 首, 抒情诗, 多为, 名篇, 佳作,...
3         [长发, 腰, 娶, 可好, 走过, 唐诗, 路过, 宋词, 相约, 江南, 美文, 佳景,...
4         [公司债务, 融资, 相关, 金融, 系统, 本书, 经济, 生活, 热点, 前沿, 公司债...
                                ...                        
206311    [四个, 学习, 读本, 首次, 权威, 定义, 五论, 协调, 推进, 四个, 评论员, ...
206312    [湖北省, 湖泊, 志, 下册, 全, 本书, 收录, 湖北省, 中型, 湖泊, 小型, 湖...
206313    [年, 计算机专业, 基础, 综合, 考试, 真题, 思路, 分析, 本书, 计算机专业, ...
206314    [野生, 作家, 访谈录, 写作, 现场, 本书, 收录, 文章, 界面, 文化频道, 记者...
206315    [西洋, 政治, 思想史, 中世纪, 篇, 本书, 欧洲, 日耳曼人, 政治, 思想, 萌发...
Name: queryCutRMStopWord, Length: 206316, dtype: object

In [15]:
import random
def Find_embedding_with_windows(embedding_matrix):
    '''
    函数说明：该函数用于获取在大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    参数说明：
        - embedding_matrix：样本中所有词构成的词向量矩阵
    return: result_list 返回拼接而成的一维词向量
    '''
        
    '''
    embedding_matrix
    
    [[0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0]]
    
    
    '''
    result_list = []
    
    size = [2, 3, 4]
    k = random.choice([2, 3, 4])
    op = random.choice([1, 2])  # 1:ave, 2: max
    # ToDo:
    # 由于之前抽取的特征并没有考虑词与词之间交互对模型的影响，
    # 对于分类模型来说，贡献最大的不一定是整个句子， 可能是句子中的一部分， 如短语、词组等等。 
    # 在此基础上我们使用大小不同的滑动窗口(k=[2, 3, 4])， 然后进行平均或取最大操作。
    
    
    for k in size:
        a = np.zeros(shape = embedding_matrix.shape)
        for i in range(embedding_matrix.shape[0] - k + 1):
            a[i] = embedding_matrix[i:i+k].mean(axis=0)
        result_list.extend(a.max(axis=0).tolist())
        
    return result_list

In [16]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [17]:
def Find_Label_embedding(word_matrix, label_embedding):
    '''
    函数说明：获取到所有类别的 label embedding， 与输入的 word embedding 矩阵相乘， 对其结果进行 softmax 运算，
            对 attention score 与输入的 word embedding 相乘的结果求平均或者取最大
            可以参考论文《Joint embedding of words and labels》获取标签空间的词嵌入
    parameters:
    -- example_matrix(np.array 2D): denotes the matrix of words embedding
    -- embedding(np.array 2D): denotes the embedding of all label in data
    return: (np.array 1D) the embedding by join label and word
    '''
    result_embedding=[]
    
    
    '''
    比如一条数据的输入的是9*300(sequence length, embedding dim)，label的数量是33，得到的attention score为（9,1），
    然后word embedding 和attention,进行np.multipy(word_embedding.T，attention )，得到（300,9）的输出，
    然后再进行avg/max，得到(300,)特征向量，和别的特征进行拼接
    '''
    
    # To_Do 
      # 第一步：基于consin相似度计算word embedding向量与label embedding之间的相似度
    
    matrix = np.dot(word_matrix, label_embedding.T)   #(9*33)
    res_matrix = []
    
    # 第二步：通过softmax获取注意力分布    (9*1)
    
    k=3 # window size
    
    for i in range(matrix.shape[1] - k + 1):
        tmp_matrix = matrix[:, i:i+k].mean(axis=1).tolist()
#         print(tmp_matrix)
        res_matrix.append(tmp_matrix)
        
    res_matrix = np.asarray(res_matrix)
    res_matrix = res_matrix.max(axis=0)
    attention = softmax(res_matrix)
                          
    # 第三步：将求得到的注意力分布与输入的word embedding相乘，并对结果进行最大化或求平均 (300, )
    
    result_embedding = (word_matrix.T * attention).T.max(axis=0)
    
    
    return result_embedding
    
   

In [18]:
def sentence2vec(query):
    '''
    函数说明：联合多种特征工程来构造新的样本表示，主要通过以下三种特征工程方法
            第一：利用word-embedding的average pooling和max-pooling
            第二：利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作
            第二：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑
            另外，对于词向量超过预定义的长度则进行截断，小于则进行填充
    参数说明：
    - query:数据集中的每一个样本
    return: 返回样本经过特征工程之后得到的词向量
    '''
    global max_length
    arr = []
    # 加载fast_embedding,w2v_embedding
    global fast_embedding, w2v_embedding
    fast_arr = np.array([fast_embedding.wv.get_vector(s)
                         for s in query if s in fast_embedding.wv.vocab.keys()])
    
    # 在fast_arr下滑动获取到的词向量
    if len(fast_arr) > 0:
        windows_fastarr = np.array(Find_embedding_with_windows(fast_arr))
        result_attention_embedding = Find_Label_embedding(
            fast_arr, label_embedding)
        
        
    else:# 如果样本中的词都不在字典，则该词向量初始化为0
        # 这里300表示训练词嵌入设置的维度为300
        windows_fastarr = np.zeros(300) 
        result_attention_embedding = np.zeros(300)

    fast_arr_max = np.max(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)
    fast_arr_avg = np.mean(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)

    fast_arr = np.hstack((fast_arr_avg, fast_arr_max))
    # 将多个embedding进行横向拼接
    arr = np.hstack((np.hstack((fast_arr, windows_fastarr)),
                     result_attention_embedding))
    global sentence_max_length
    # 如果样本的维度大于指定的长度则需要进行截取或者拼凑,
#     result_arr = arr[:sentence_max_length] if len(arr) > sentence_max_length else np.hstack((
#         arr, np.zeros(int(sentence_max_length-len(arr)))))
    if len(arr) > sentence_max_length:
        indice = random.sample(range(0, len(arr)), sentence_max_length)
        indice.sort()
        result_arr = arr[indice]
        
    else: 
        result_arr = np.hstack((arr, np.zeros(int(sentence_max_length-len(arr)))))
    
#     print(result_arr.shape)
    
    return result_arr

In [19]:
def Dimension_Reduction(Train, Test):
    '''
    函数说明：该函数通过PCA算法对样本进行降维，由于目前维度不是特别高 ，可以选择不降维。
    参数说明：
    - Train: 表示训练数据集
    - Test: 表示测试数据集
    Return: 返回降维之后的数据样本
    '''
    global max_length
    #  To_Do 
    # 特征选择，由于经过特征工程得到的样本表示维度很高，因此需要进行降维 max_length表示降维之后的样本最大的维度。
    # 这里通过PCA方法降维
    
    estimator = PCA(n_components=max_length)
    pca_train = estimator.fit_transform(Train)
    pca_test = estimator.fit_transform(Test)
    return pca_train, pca_test

In [20]:
def Find_Embedding():
    '''
    函数说明：该函数用于获取经过特征工程之后的样本表示
    Return:训练集特征数组(2D)，测试集特征数组(2D)，训练集标签数组（1D）,测试集标签数组（1D）
    '''
    print("获取样本表示中...")
    min_max_scaler = preprocessing.MinMaxScaler()
    Train_features2 = min_max_scaler.fit_transform(
        np.vstack(train["queryCutRMStopWord"].apply(sentence2vec)))
    Test_features2 = min_max_scaler.fit_transform(
        np.vstack(test["queryCutRMStopWord"].apply(sentence2vec)))
    print("获取样本词表示完成")
    # 通过PCA对样本表示进行降维
    Train_features2, Test_features2 = Dimension_Reduction(
        Train=Train_features2, Test=Test_features2)
    Train_label2 = train["labelIndex"]
    Test_label2 = test["labelIndex"]

    print("加载训练好的词向量")
    print("Train_features.shape =", Train_features2.shape)
    print("Test_features.shape =", Test_features2.shape)
    print("Train_label.shape =", Train_label2.shape)
    print("Test_label.shape =", Test_label2.shape)

    return Train_features2, Test_features2, Train_label2, Test_label2


In [21]:
def Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, model_name):
    '''
    函数说明：直接输出训练集和测试在模型上的准确率
    参数说明：
        - Train_label: 真实的训练集标签（1D）
        - Test_labelb: 真实的测试集标签（1D）
        - Train_predict_label: 模型在训练集上的预测的标签(1D)
        - Test_predict_label: 模型在测试集上的预测标签（1D）
        - model_name: 表示训练好的模型
    Return: None
    '''
    # ToDo
    
    # 通过调用metrics.accuracy_score计算训练集和测试集上的准确率  Search_Flag+
    print(model_name+'_'+'Train accuracy %s' % metrics.accuracy_score(Train_label, Train_predict_label))
    # 输出测试集的准确率   Search_Flag+
    print(model_name+'_'+'Test accuracy %s' %  metrics.accuracy_score(Test_label, Test_predict_label))

# ToDo
第一步：完成LGBMClassifier模型中参数  
第二步：根据joblib.dump保存训练好的模型

In [26]:
def Grid_Train_model(Train_features, Test_features, Train_label, Test_label):
    '''
    函数说明：基于网格搜索优化的方法搜索模型最优参数，最后保存训练好的模型
    参数说明：
        - Train_features: 训练集特征数组（2D）
        - Test_features: 测试集特征数组（2D）
        - Train_label: 真实的训练集标签 (1D)
        - Test_label: 真实的测试集标签（1D）
    Return: None
    '''
#     parameters = {
#         'max_depth': [5, 10, 15, 20, 25],
#         'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
#         'n_estimators': [100, 500, 1000, 1500, 2000],
#         'min_child_weight': [0, 2, 5, 10, 20],
#         'max_delta_step': [0, 0.2, 0.6, 1, 2],
#         'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
#         'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
#         'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
#         'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
#         'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]
#     }
    
    
    parameters = {
        'max_depth': [1, 5],
        'learning_rate': [0.01, 0.05],
        'n_estimators': [100, 1000],
        'min_child_weight': [0, 6],
        'max_delta_step': [0, 0.6],
        'subsample': [0.6, 0.75],
#         'colsample_bytree': [0.5, 0.8],
        'reg_alpha': [0, 0.5],
#         'reg_lambda': [0.2, 1],
#         'scale_pos_weight': [0.2, 0.8]
    }
    # 定义分类模型列表，这里仅使用LightGBM模型
    models = [
        lgb.LGBMClassifier(),
    ]
    # 遍历模型
    for model in models:
        model_name = model.__class__.  __name__
        gsearch = GridSearchCV(
            model, param_grid=parameters, scoring='accuracy', cv=3)
        gsearch.fit(Train_features, Train_label)
        # 输出最好的参数-
        print("Best parameters set found on development set:{}".format(
            gsearch.best_params_))
        Test_predict_label = gsearch.predict(Test_features)
        Train_predict_label = gsearch.predict(Train_features)
        Predict(Train_label, Test_label,
                Train_predict_label, Test_predict_label, model_name)
    # 保存训练好的模型
    joblib.dump('#ToDo'+'.pkl')


主函数,先求训练集和测试集的词向量，然后根据Grid搜索来找到最佳参数的分类模型

已完成特征提取

In [25]:
Train_features, Test_features, Train_label, Test_label = Find_Embedding()

获取样本表示中...
获取样本词表示完成
加载训练好的词向量
Train_features.shape = (206316, 500)
Test_features.shape = (29474, 500)
Train_label.shape = (206316,)
Test_label.shape = (29474,)


KeyboardInterrupt: 

没训练出来 不再调试模型了

In [ ]:
Grid_Train_model(Train_features=Train_features, Test_features=Test_features,Train_label=Train_label, Test_label=Test_label)

In [ ]:
import pickle

file_x_1 = open('Train_features.pickle', 'wb')
pickle.dump(Train_features, file_x_1)
file_x_1.close()

file_x_2 = open('Test_features.pickle', 'wb')
pickle.dump(Test_features, file_x_2)
file_x_2.close()

file_x_3 = open('Train_label.pickle', 'wb')
pickle.dump(Train_label, file_x_3)
file_x_3.close()

file_x_4 = open('Test_label.pickle', 'wb')
pickle.dump(Test_label, file_x_4)
file_x_4.close()

In [ ]:
with open('Train_features.pickle', 'rb') as file:
    Train_features =pickle.load(file)
    
with open('Test_features.pickle', 'rb') as file:
    Test_features =pickle.load(file)
    
with open('Train_label.pickle', 'rb') as file:
    Train_label =pickle.load(file)
    
with open('Test_label.pickle', 'rb') as file:
    Test_label =pickle.load(file)